In [43]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [207]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [237]:

tX_list, ids_list = separate_dataset(tX, ids)
print(tX_list[1].shape)


alpha = 0.98
for i in range(6):
    dist_fr_mean = np.std(tX_list[i], axis = 0)/(np.sqrt(1-alpha))
    mean = np.average(tX_list[i], axis = 0)
    print(tX_list[i].shape)
    tX_list[i] = np.where(np.abs(tX_list[i]-mean) > dist_fr_mean, np.nan,tX_list[i])
    tX_list[i] = tX_list[i][~np.isnan(tX_list[i]).any(axis=1)]
    print(tX_list[i].shape)


# for i in range(1):
#     a = np.quantile(tX_list[0], alpha/2, axis = 0)
#     b = np.quantile(tX_list[0], 1-(alpha/2), axis = 0)
#     tX_list[0] = np.where(np.logical_or(tX_list[0]<a, tX_list[0]>b), np.nan,tX_list[0])
#     print((tX_list[0][~np.isnan(tX_list[0]).any(axis=1)]).shape)

(73790, 18)


In [120]:
#Check data
tX_list, ids_list = separate_dataset(tX, ids)
index=[]
for seT in range(len(tX_list)):
    for indexX, column in enumerate(tX_list[seT].T):
        x_= np.mean(column)
        for indexY, column2 in enumerate(tX_list[seT].T):
            y_= np.mean(column2)
            num=np.dot((column - x_),(column2 - y_))
            den= np.sqrt(np.dot(column - x_,column - x_)*np.dot(column2 - y_,column2 - y_))
            corr= num/den
            if corr> 0.85 and indexX!= indexY and indexX< indexY:
                index.append([seT,indexX, indexY])

print(index)
      
def datamodif(tX_list, log=False):
    tX_list[0]=np.delete(tX_list[0], [4], axis=1)
    print(tX_list[0].shape)
    long_tail = [0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 15]
    tX_list[0][:, long_tail]= np.sqrt(tX_list[0][:, long_tail])
    if log:
        normalize(tX_list[0])
    else:
        standardize(tX_list[0])

    tX_list[1]=np.delete(tX_list[1], [2,5], axis=1)
    print(tX_list[1].shape)
    long_tail = [0, 1, 4, 5, 7, 10, 13, 7, 13, 15]
    tX_list[1][:, long_tail]= np.sqrt(tX_list[1][:, long_tail])
    #tX_list[1][:, 6]= np.where(tX_list[1][:, 6]> 0.5, 1, 0)
    if log:
        normalize(tX_list[1])
    else:
        standardize(tX_list[1])

    tX_list[2]=np.delete(tX_list[2], [20], axis=1)
    print(tX_list[2].shape)
    long_tail = [0, 1, 2, 3, 4, 5, 6, 8, 11, 14, 16, 17]
    tX_list[2][:, long_tail]= np.sqrt(tX_list[2][:, long_tail])
    #tX_list[2][:, 7]= np.where(tX_list[2][:, 7]> 0.5, 1, 0)
    if log:
        normalize(tX_list[2])
    else:
        standardize(tX_list[2])

    tX_list[3]=np.delete(tX_list[3], [3,6,18], axis=1)
    print(tX_list[3].shape)
    long_tail = [0, 1, 2, 3, 4, 5, 7, 10, 13, 15, 18]
    tX_list[3][:, long_tail]= np.sqrt(tX_list[3][:, long_tail])
    #tX_list[3][:, 6]= np.where(tX_list[3][:, 6]> 0.5, 1, 0)
    if log:
        normalize(tX_list[3])
    else:
        standardize(tX_list[3])

    tX_list[4]=np.delete(tX_list[4], [27], axis=1)
    print(tX_list[4].shape)
    long_tail = [0, 1, 2, 3, 4, 6, 7, 8, 9, 12, 15, 18, 20, 21]
    tX_list[4][:, long_tail]= np.sqrt(tX_list[4][:, long_tail])
    #tX_list[4][:, 11]= np.where(tX_list[4][:, 11]> 0.5, 1, 0)
    #tX_list[4][:, 10]= np.where(tX_list[4][:, 10]> 0.5, 1, 0)
    if log:
        normalize(tX_list[4])
    else:
        standardize(tX_list[4])

    tX_list[5]=np.delete(tX_list[5], [2,9,28], axis=1)
    print(tX_list[5].shape)
    long_tail = [0, 1, 2, 3, 4, 6, 7, 8, 11, 14, 17, 19, 20, 23]
    tX_list[5][:, long_tail]= np.sqrt(tX_list[5][:, long_tail])
    #tX_list[5][:, 9]= np.where(tX_list[5][:, 9]> 0.5, 1, 0)
    #tX_list[5][:, 10]= np.where(tX_list[5][:, 10]> 0.5, 1, 0)
    if log:
        normalize(tX_list[5])
    else:
        standardize(tX_list[5])

    return tX_list

#import seaborn as sns
#for indexX, column in enumerate(tX_list[5].T):
#    sns.displot(column)


[[0, 2, 4], [1, 0, 2], [1, 3, 5], [2, 17, 20], [3, 0, 2], [3, 3, 6], [3, 3, 18], [3, 3, 21], [3, 6, 18], [3, 6, 21], [3, 18, 21], [4, 8, 27], [4, 21, 27], [5, 0, 2], [5, 9, 21], [5, 9, 22], [5, 9, 28], [5, 21, 28], [5, 22, 28]]


In [186]:
#Load Data
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)

tX_list = datamodif(tX_list)
tX_test_list = datamodif(tX_test_list)

(26123, 16)
(73790, 16)
(7562, 20)
(69982, 19)
(4429, 27)
(68114, 26)
(59263, 16)
(168195, 16)
(17243, 20)
(158095, 19)
(9982, 27)
(155460, 26)


In [187]:
#Grid Search Ridge

function = ridge_regression
degree_vec = []
lambda_vec = []


for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, _ = grid_search(y_list[i], tX_list[i], ridge_regression, log = False, k_fold = 4, degrees = range(1, 20), lambdas = np.logspace(-13, -5, 20), gammas = np.arange(1))
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Loss: {lo}'.format(lo = rmse_te))

Dataset part 0:
     Best degree: 5
     Best lambda: 6.951927961775591e-13
     Loss: 0.41281470964633366
Dataset part 1:
     Best degree: 19
     Best lambda: 2.06913808111479e-07
     Loss: 0.7616515593061501
Dataset part 2:
     Best degree: 4
     Best lambda: 7.847599703514607e-08
     Loss: 0.5231766428569052
Dataset part 3:
     Best degree: 7
     Best lambda: 3.359818286283788e-11
     Loss: 0.8025224676686908
Dataset part 4:
     Best degree: 9
     Best lambda: 1.43844988828766e-06
     Loss: 0.5451902719212371
Dataset part 5:
     Best degree: 7
     Best lambda: 2.6366508987303554e-13
     Loss: 0.7539102873274455


In [188]:
#Train

weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], False, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

In [189]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, False) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

In [178]:
#Load Data for Log
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

from implementations import *



(26123, 16)
(73790, 16)
(7562, 20)
(69982, 19)
(4429, 27)
(68114, 26)
(59263, 16)
(168195, 16)
(17243, 20)
(158095, 19)
(9982, 27)
(155460, 26)


In [179]:
function = reg_logistic_regression
degree_vec = []
lambda_vec = []
gamma_vec= []

for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search(y_list[i], tX_list[i], reg_logistic_regression, log = False, k_fold = 4, degrees = range(1,3), lambdas = np.logspace(-15, -6, 8), gammas = np.logspace(-4, 0, 5))
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Best gamma: {g}'.format(g = BestGamma))
    print('     Loss: {lo}'.format(lo = rmse_te))

Dataset part 0:
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold
treshold


KeyboardInterrupt: 

In [173]:
#Train Logistic
from implementations import *

weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 5000

for i in range(6):
    initial_w= np.zeros((tX_list[i].shape[1], 1))
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], False, tX_test_list[i])
    l, w = reg_logistic_regression(y_list[i], tX_list[i], initial_w, max_iter, lambda_vec[i], gamma_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

treshold
treshold
treshold
treshold
treshold
treshold


In [176]:
# Eval Logistic

y_pred_list = separated_eval(weights_list, mat_tX_test_list, True) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

In [14]:
#Importation of train and test data:

from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

#Separate Data Set
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)



#Grid Search of param
function = ridge_regression
degree_vec = []
lambda_vec = []
for i in range(3):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda = grid_search(y_list[i], tX_list[i], function, True)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Loss: {lo}'.format(lo = rmse_te))



#Training
weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(3):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], True, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

Dataset part 0:


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


     Best degree: 1
     Best lambda: 0.0008733261623828437
     Loss: 0.7380547288266697
Dataset part 1:
     Best degree: 5
     Best lambda: 0.024118646996409948
     Loss: 0.8058598609702398
Dataset part 2:
     Best degree: 4
     Best lambda: 0.005817091329374358
     Loss: 0.7826367955518109


In [6]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

#Separate Data Set
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)

degree_vec = []
lambda_vec = []
gamma_vec = []
for i in range(3):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search_for_log_reg(y_list[i], tX_list[i], False)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Best gamma: {g}'.format(g = BestGamma))
    print('     Loss: {lo}'.format(lo = rmse_te))

weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 10000


for i in range(3):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], False, tX_test_list[i])
    initial_w = np.zeros((mat_tX.shape[1], 1))
    l, w = logistic_regression_penalized_gradient_descent(y_list[i], mat_tX, initial_w, max_iter, gamma_vec[i], lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

Dataset part 0:
Done Lambda
Done Lambda
Done Lambda
Done Lambda
Done Lambda


KeyboardInterrupt: 

In [17]:
#Log Reg Pen


from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

print("Data loaded")


#Separate Data Set
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)
print("Data separated")

degree_vec = [1 , 4 , 5]
lambda_vec = [1e-05 , 1e-04 , 1e-04]
gamma_vec = [8.65e-10 , 1.9e-9, 1e-9]

weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 10000

for i in range(3):
    print("loop")
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], False, tX_test_list[i])
    initial_w = np.zeros((mat_tX.shape[1], 1))
    l, w = logistic_regression_penalized_gradient_descent(y_list[i], mat_tX, initial_w, max_iter, gamma_vec[i], lambda_vec[i])
    print(l)
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

Data loaded
Data separated
loop
-64730.23741869711
loop
-63383.06999684557
loop
-181.017636445889


## Generate predictions and save ouput in csv format for submission:

In [105]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_poly=build_poly(tX_test,BestDeg)

In [106]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [18]:
#Submission with separate data, grid search and ridge of log

OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

# mat_tX_test_list = build_poly_separated(tX_test_list, 3)            #data augmentation
y_pred_list = separated_eval(weights_list, mat_tX_test_list)        #prediction

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)